In [30]:
import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser
import matplotlib.pyplot as plt


In [31]:
config = configparser.ConfigParser()
config.read('escec2.cfg')

['escec2.cfg']

In [32]:
RDS_HOST = 'proyectof.csgdbjf8hjye.us-east-1.rds.amazonaws.com'
#RDS_HOST = 'banco-db-v2.cu1ulzl8g9j8.us-east-1.rds.amazonaws.com'
#RDS_HOST=config.get('DW', 'RDS_HOST')
#RDS_HOST
print(RDS_HOST)

proyectof.csgdbjf8hjye.us-east-1.rds.amazonaws.com


In [33]:
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""  

### se crean las tablas

In [34]:
sql_query = 'SELECT * FROM dim_fecha;'
df_fecha = pd.read_sql(sql_query, postgres_driver)
df_fecha.head()

,id_fecha,fecha,dia_semana,mes
0,0,2023-06-14,Saturday,October
1,1,2023-05-02,Thursday,January
2,2,2022-12-07,Friday,December
3,3,2022-05-12,Saturday,June
4,4,2024-03-31,Saturday,November


In [35]:
sql_query = 'SELECT * FROM dim_cajeros;'
df_cajeros = pd.read_sql(sql_query, postgres_driver)
df_cajeros.head()

,id_cajero,nombre,apellido,id_cargo
0,0,Daniel Collins,Johnson,"{206,710,643,922,236}"
1,1,Christine Wilkerson,Martinez,"{877,930,399,375,721}"
2,2,Rebecca Wilson,Cummings,"{766,576,262,512,251}"
3,3,Cynthia Logan,Tyler,"{330,464,590,517,59}"
4,4,Brandon Kim,Rodriguez,"{602,572,629,412,943}"


In [36]:
sql_query = 'SELECT * FROM dim_pagos;'
df_pago = pd.read_sql(sql_query, postgres_driver)
df_pago.head()

,id_pago,metodo_pago
0,997,tarjeta
1,198,tarjeta
2,402,tarjeta
3,838,transferencia
4,637,contra entrega


In [37]:
sql_query = 'SELECT * FROM dim_tienda;'
df_tienda = pd.read_sql(sql_query, postgres_driver)
df_tienda.head()

,id_tienda,nombre_tienda,id_gerente
0,0,Manning LLC,"{83435,56388,69934,52281,47188}"
1,1,"Ball, Reed and Rice","{38307,13733,81287,89481,4078}"
2,2,"Knox, Day and West","{32036,68203,74778,27543,3940}"
3,3,Mitchell and Sons,"{69300,70041,39174,28636,50247}"
4,4,"Stone, Wilson and Bishop","{55507,47784,26440,92934,93831}"


In [41]:
sql_query = 'SELECT * FROM dim_promocion;'
df_promocion = pd.read_sql(sql_query, postgres_driver)
df_promocion.head()

,id_promocion,codigo_promocional
0,79,57
1,29,29
2,94,75
3,59,20
4,45,81


In [42]:
sql_query = 'SELECT * FROM dim_productos;'
df_producto = pd.read_sql(sql_query, postgres_driver)
df_producto.head()

,id_producto,sku_numero,producto,marca,categoria_producto
0,5020,SKU-0104,Book skill positive.,Williams-Bishop,least
1,6818,SKU-4225,Focus newspaper start tough.,Mcmillan Inc,even
2,1972,SKU-1183,Wear fly place.,Mcbride Group,positive
3,3489,SKU-5093,Pay policy.,Guzman-Mora,protect
4,2096,SKU-9854,Support ago.,Moore LLC,Mrs


In [13]:
sql_query = 'SELECT * FROM transacciones;'
df_transacciones = pd.read_sql(sql_query, postgres_driver)
df_transacciones.head()

,id_transaccion,cantidad_vendida,monto,id_fecha,id_cajero,id_tienda,id_producto,id_promocion,id_pago,id_direccion
0,45316,749,7964.77,929,57,55,7074,50,397,97
1,766638,177,7128.02,182,49,2,8192,94,113,90
2,925603,600,2632.90,801,27,62,579,46,987,14
3,988076,442,4473.95,692,29,71,3389,38,790,7
4,630855,139,1420.78,387,55,48,5521,23,736,34


### se hace la union de la tabla transacciones con la tabla fecha

In [14]:
df_fecha_transacciones = df_fecha.merge(df_transacciones, on='id_fecha', how='inner')
df_fecha_transacciones.head()

,id_fecha,fecha,dia_semana,mes,id_transaccion,cantidad_vendida,monto,id_cajero,id_tienda,id_producto,id_promocion,id_pago,id_direccion
0,2,2022-12-07,Friday,December,968892,650,4298.73,48,15,536,59,468,88
1,3,2022-05-12,Saturday,June,309846,700,6742.38,1,88,9241,63,736,90
2,3,2022-05-12,Saturday,June,245228,337,3942.83,68,67,3664,20,560,59
3,3,2022-05-12,Saturday,June,162260,964,2034.17,28,57,9570,40,707,90
4,4,2024-03-31,Saturday,November,417138,848,8168.26,2,69,2565,60,42,5


### se repite la misma union, pero se eleije solo algunas columnas

In [15]:
df_fechas_transacciones = pd.merge(df_fecha[['id_fecha','fecha']],
                                    df_transacciones[['id_fecha','cantidad_vendida','monto']], on='id_fecha')
df_fechas_transacciones

,id_fecha,fecha,cantidad_vendida,monto
0,2,2022-12-07,650,4298.73
1,3,2022-05-12,700,6742.38
2,3,2022-05-12,337,3942.83
3,3,2022-05-12,964,2034.17
4,4,2024-03-31,848,8168.26
...,...,...,...,...
1932,1120,2021-07-17,196,5613.21
1933,1120,2021-07-17,853,3379.27
1934,1121,2024-03-02,707,5978.44
1935,1121,2024-03-02,393,5475.76


### esta vez se unen tres tablas y se eligen las columnas de interes, se une la tabla de fechas, pagos y transacciones y se elije a interes las tablas a mostrar

In [16]:

# unir tabla de transacciones con tabla de fechas
hechos_fecha = df_transacciones.merge(df_fecha, on='id_fecha')

# unir tabla de transacciones con tabla de pagos
tienda_pago_hechos = hechos_fecha.merge(df_pago, on='id_pago')

# seleccionar columnas deseadas
df_fecha_df_pago_df_transacciones = tienda_pago_hechos[['mes', 'monto', 'metodo_pago']]

df_fecha_df_pago_df_transacciones

,mes,monto,metodo_pago
0,July,7964.77,contra entrega
1,March,4343.05,contra entrega
2,February,1423.92,contra entrega
3,August,9419.23,contra entrega
4,February,5024.32,contra entrega
...,...,...,...
2053,January,1665.00,transferencia
2054,April,2603.30,transferencia
2055,July,1085.70,transferencia
2056,September,5049.46,transferencia


### ¿cual es el metodo de pago mas comun?

In [51]:
# obtener la frecuencia de cada valor en la columna "metodo_pago"
frecuencias = df_fecha_df_pago_df_transacciones['metodo_pago'].value_counts()

# seleccionar el valor con la frecuencia más alta
se_repite = frecuencias.index[0]

# imprimir el valor más común
print('El metodo de pago mas comun es:', se_repite)


El metodo de pago mas comun es: tarjeta


### ¿en que mes se vende mas?

In [54]:
# agrupar por mes y sumar los montos de todas las transacciones de cada mes
ventas_por_mes = df_fecha_df_pago_df_transacciones.groupby('mes')['monto'].sum()

# seleccionar el mes con la suma más alta
mes_con_mas_ventas = ventas_por_mes.idxmax()

# imprimir el mes con la suma más alta
print('El mes con más ventas fue:', mes_con_mas_ventas,)


El mes con más ventas fue: February


### ¿y de cuanto fue la venta?

In [55]:
# agrupar por mes y sumar los montos de todas las transacciones de cada mes
ventas_por_mes = df_fecha_df_pago_df_transacciones.groupby('mes')['monto'].sum()

# seleccionar el mes con la suma más alta y su valor de venta correspondiente
mes_con_mas_ventas = ventas_por_mes.idxmax()
venta_mes_con_mas_ventas = ventas_por_mes.max()

# imprimir el mes con la suma más alta y su valor de venta correspondiente
print(f'El mes con más ventas fue {mes_con_mas_ventas} y su venta total fue de {venta_mes_con_mas_ventas} dólares.')


El mes con más ventas fue February y su venta total fue de 1117613.79 dólares.


### ¿que cajero vendio mas? 

In [57]:
# unir tabla de transacciones con tabla de fechas
transaccion_fecha = df_transacciones.merge(df_fecha, on='id_fecha')

# unir tabla de transacciones con tabla de pagos
cajeros_transaccion_fecha = transaccion_fecha.merge(df_cajeros, on='id_cajero')

# seleccionar columnas deseadas
df_fecha_df_cajeros_df_transacciones = cajeros_transaccion_fecha[['nombre', 'monto', 'fecha']]

df_fecha_df_cajeros_df_transacciones

,nombre,monto,fecha
0,Nicholas Washington,7964.77,2022-03-05
1,Nicholas Washington,3308.31,2021-10-13
2,Nicholas Washington,3014.65,2023-03-08
3,Nicholas Washington,8370.61,2023-02-13
4,Nicholas Washington,8993.52,2023-12-19
...,...,...,...
1932,Michael Gonzalez,7582.58,2023-06-15
1933,Michael Gonzalez,3906.34,2023-03-17
1934,Michael Gonzalez,8384.72,2022-07-11
1935,Michael Gonzalez,9574.09,2022-10-30


In [58]:
# agrupar por cajero y sumar los montos de todas las transacciones de cada cajero
ventas_por_cajero = df_fecha_df_cajeros_df_transacciones.groupby('nombre')['monto'].sum()

# seleccionar el cajero con la suma más alta
cajero_con_mas_ventas = ventas_por_cajero.idxmax()

# imprimir el cajero con la suma más alta
print('El cajero que más vendió fue:', cajero_con_mas_ventas)


El cajero que más vendió fue: Whitney Bryan


In [59]:
# agrupar por cajero y sumar los montos de todas las transacciones de cada cajero
ventas_por_cajero = df_fecha_df_cajeros_df_transacciones.groupby('nombre')['monto'].sum()

# seleccionar el cajero con la suma más alta y su valor de venta correspondiente
cajero_con_mas_ventas = ventas_por_cajero.idxmax()
venta_cajero_con_mas_ventas = ventas_por_cajero.max()

# imprimir el cajero con la suma más alta y su valor de venta correspondiente
print(f'El cajero que más vendió fue {cajero_con_mas_ventas} y su venta total fue de {venta_cajero_con_mas_ventas} dólares.')


El cajero que más vendió fue Whitney Bryan y su venta total fue de 218749.74 dólares.
